### RNN Assignment

import needed libraries

In [29]:
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.layers import GRU, Dropout
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense,Bidirectional
import keras
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from tensorflow.keras.applications import VGG19
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.models import Model

import data

get max length

In [16]:
max_len=0
all_data_x=[]
all_data_y=[]
for subject_id in range(1, 11):
    file_name = f"mHealth_subject{subject_id}.log"
    
    with open(file_name, "r", encoding="utf8") as file:
        count={}
        prev_label=0
        data=[]
        for line in file:
            tokens = line.split()
            label = int(tokens[-1])
            if(label!= 0):
                if(prev_label!=label and prev_label!=0):
                    all_data_x.append(data)
                    all_data_y.append(prev_label)
                    count[prev_label]=len(data)
                           
                columns=len(tokens)-1
                if(prev_label !=label):
                    data+=[tokens[:-1]]
                prev_label=label
            
        # print(count)
        all_data_x.append(data)
        all_data_y.append(prev_label)
        count[prev_label]=len(data)
        for value in count.values():
            if(value>max_len):
                max_len=value
                

print the maximum length

In [17]:
print("maximun lenth of data =",max_len)

maximun lenth of data = 12


In [18]:
max_column=23 ## using question
num_labels=12
data_count=10*num_labels

In [19]:
def get_em_data(data,max_len,max_column):
    embedding_data= np.zeros((len(data), max_len, max_column))
    
    for index in range(len(data)):
        num_times=len(data[index])
        for i in range(num_times):
            embedding_data[index][i]= np.array(data[index][i], dtype=float)
    return embedding_data

Convert into numpy array

In [20]:
y_array=np.array(all_data_y)

print a sample array

In [21]:
y_array

array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  5,  5,  1,  2,  3,  4,
        6,  7,  8, 10, 11, 12,  9,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11,
       12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,
        4,  6,  7,  8,  9, 10, 11, 12,  5,  1,  2,  3,  4,  6,  7,  8,  9,
       10, 11, 12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  1,
        3,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  6,  7,
        8,  9, 10, 11, 12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11,
       12])

In [22]:
x_array=get_em_data(all_data_x,max_len +1,max_column)

convert to categorical

In [23]:
y_convert =y_array-1
y_array_c = to_categorical(y_convert)


create a model

In [24]:
print(max_column)

23


This is the model GRU

In [ ]:


# Load the pre-trained model and tokenizer (e.g., BERT)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
pretrained_model = TFAutoModel.from_pretrained(model_name)

# Remove the original output layers
output_layer = pretrained_model.layers[-1]
pretrained_model = Model(pretrained_model.input, output_layer.input)

# Add new layers for your target task
x = Dense(units=num_labels, activation='softmax')(pretrained_model.output)

# Create a new model
model = Model(pretrained_model.input, x)



Define losss function

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

Fit the model

In [ ]:
result=model.fit(x_array,y_array_c,epochs=20,batch_size=20,validation_split=0.2, shuffle=True)
#batch_size=20,validation_split=0.2

Epoch 1/20
5/5 [==============================] - 16s 828ms/step - loss: 2.9101 - accuracy: 0.0312 - val_loss: 2.5247 - val_accuracy: 0.0833
Epoch 2/20
5/5 [==============================] - 2s 356ms/step - loss: 2.7298 - accuracy: 0.0312 - val_loss: 2.5427 - val_accuracy: 0.0833
Epoch 3/20
5/5 [==============================] - 2s 319ms/step - loss: 2.6311 - accuracy: 0.0521 - val_loss: 2.5153 - val_accuracy: 0.0833
Epoch 4/20
5/5 [==============================] - 2s 326ms/step - loss: 2.5476 - accuracy: 0.0417 - val_loss: 2.5080 - val_accuracy: 0.0833
Epoch 5/20
5/5 [==============================] - 2s 318ms/step - loss: 2.5438 - accuracy: 0.0833 - val_loss: 2.4905 - val_accuracy: 0.0833
Epoch 6/20
5/5 [==============================] - 2s 305ms/step - loss: 2.5134 - accuracy: 0.0625 - val_loss: 2.4922 - val_accuracy: 0.0833
Epoch 7/20
5/5 [==============================] - 2s 318ms/step - loss: 2.5152 - accuracy: 0.0833 - val_loss: 2.4926 - val_accuracy: 0.0833
Epoch 8/20
5/5 [===

Predict some values

In [48]:
predicted = model.predict(x_array[2:8])

1/1 [==============================] - 1s 920ms/step


In [49]:
predicted[0]

array([0.09010875, 0.08765221, 0.07913466, 0.0820458 , 0.0762894 ,
       0.08171646, 0.09436854, 0.07710967, 0.08104338, 0.08866577,
       0.08502123, 0.07684417], dtype=float32)

In [50]:
predicted_labels = np.argmax(predicted, axis=1)

Find the accuracy

In [67]:
training_accuracy = result.history['accuracy']
validation_accuracy = result.history['val_accuracy']


In [68]:
print( "final training accuracy = ",training_accuracy[-1])
print( "final testing accuracy = ",validation_accuracy[-1])

final training accuracy =  0.0833333358168602
final testing accuracy =  0.0833333358168602


In [69]:
print( "maximum training accuracy = ",max(training_accuracy))
print( "maximum testing accuracy = ",max(validation_accuracy))

maximum training accuracy =  0.0833333358168602
maximum testing accuracy =  0.0833333358168602


Save the model

In [71]:
model.save("Transfer_model_1.h5")